# Introducción

La compañía de seguros Sure Tomorrow quiere resolver varias tareas con la ayuda de machine learning y te pide que evalúes esa posibilidad.
- Tarea 1: encontrar clientes que sean similares a un cliente determinado. Esto ayudará a los agentes de la compañía con el marketing.
- Tarea 2: predecir la probabilidad de que un nuevo cliente reciba una prestación del seguro. ¿Puede un modelo de predictivo funcionar mejor que un modelo dummy?
- Tarea 3: predecir el número de prestaciones de seguro que un nuevo cliente pueda recibir utilizando un modelo de regresión lineal.
- Tarea 4: proteger los datos personales de los clientes sin afectar al modelo del ejercicio anterior. Es necesario desarrollar un algoritmo de transformación de datos que dificulte la recuperación de la información personal si los datos caen en manos equivocadas. Esto se denomina enmascaramiento u ofuscación de datos. Pero los datos deben protegerse de tal manera que no se vea afectada la calidad de los modelos de machine learning. No es necesario elegir el mejor modelo, basta con demostrar que el algoritmo funciona correctamente.


# Preprocesamiento y exploración de datos

## Inicialización

In [ ]:
# Actualizar e instalar scikit-learn

!pip install scikit-learn --upgrade

In [ ]:
# Importación de librerías para análisis de datos y modelado

import numpy as np
import pandas as pd
import math

import seaborn as sns

import sklearn.linear_model
import sklearn.metrics
import sklearn.neighbors
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

from IPython.display import display

## Carga de datos

Carga los datos y haz una revisión básica para comprobar que no hay problemas obvios.

In [ ]:
# Carga de datos

try:
    df = pd.read_csv('insurance_us.csv')
except:
    df = pd.read_csv('/datasets/insurance_us.csv')

Renombramos las columnas para que el código se vea más coherente con su estilo.

In [ ]:
# Estandarización de nombres de columnas

df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [ ]:
# Selección aleatoria de 10 registros

df.sample(10)

In [ ]:
# Resumen de estructura y tipos de datos

df.info()

In [ ]:
# Cambio del tipo de edad (de float a int)

df['age'] = df['age'].astype(int)

df.info()

In [ ]:
# Descripción del df

df.describe()

In [ ]:
# Conteo de personas sin prestaciones de seguros

zeros = (df['insurance_benefits'] == 0).sum()
print('Personas sin prestraciones de seguros: ', zeros)

El conjunto de datos consta de 5,000 observaciones con una representación equitativa de hombres y mujeres, tal y como indica la media de **género** de 0,50, lo que confirma que se trata de una muestra equilibrada. 

La **edad** media de los participantes es de aproximadamente 31 años, y la mayoría de ellos tienen entre principios de los veintes y finales de los treinta años, lo que refleja una población adulta relativamente joven. 

Los **niveles de ingresos** varían moderadamente, con un ingreso medio anual de unos \\$ 39,916 y un rango que oscila entre los \\$ 5,300 y los \\$ 79,000, lo que sugiere una distribución concentrada en torno a la franja de ingresos medios-bajos, pero que incluye a algunas personas con ingresos más elevados. 

El **tamaño de los hogares** tiende a ser pequeño, con una media de poco más de un miembro por hogar, y una parte significativa de la muestra vive sola o en hogares de dos personas, como lo muestra el tamaño medio de la familia, que es de uno, y el percentil 25, que es cero. 

La **cobertura de las prestaciones del seguro es notablemente baja**, con una media de solo 0.15. La mayoría de los participantes, **4,436 personas que representan el 89%, carece de prestaciones del seguro**, lo que indica un acceso o una inscripción limitados en los programas de seguros dentro de esta población.

En general, los datos muestran una población joven y equilibrada, con ingresos modestos, hogares pequeños y una cobertura de seguro mínima.

## Análisis exploratorio de datos

Vamos a comprobar rápidamente si existen determinados grupos de clientes observando el gráfico de pares.

In [ ]:
# Visualización de histogramas y relaciones por pares

g = sns.pairplot(df, kind='hist')
g.fig.set_size_inches(12, 12)

De acuerdo, es un poco complicado detectar grupos obvios (clústeres) ya que es difícil combinar diversas variables simultáneamente (para analizar distribuciones multivariadas). Ahí es donde LA y ML pueden ser bastante útiles.

# Tarea 1. Clientes similares

En el lenguaje de ML, es necesario desarrollar un procedimiento que devuelva los k vecinos más cercanos (objetos) para un objeto dado basándose en la distancia entre los objetos.
Es posible que quieras revisar las siguientes lecciones (capítulo -> lección)- Distancia entre vectores -> Distancia euclidiana
- Distancia entre vectores -> Distancia Manhattan

Para resolver la tarea, podemos probar diferentes métricas de distancia.

Escribe una función que devuelva los k vecinos más cercanos para un $n^{th}$ objeto basándose en una métrica de distancia especificada. A la hora de realizar esta tarea no debe tenerse en cuenta el número de prestaciones de seguro recibidas.
Puedes utilizar una implementación ya existente del algoritmo kNN de scikit-learn (consulta [el enlace](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)) o tu propia implementación.
Pruébalo para cuatro combinaciones de dos casos- Escalado
  - los datos no están escalados
  - los datos se escalan con el escalador [MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)
- Métricas de distancia
  - Euclidiana
  - Manhattan

In [ ]:
# Función para obtener los k vecinos cercanos usando métrica de distancia

def get_knn(df, n, k, metric):
    
    """
    Devuelve los k vecinos más cercanos

    :param df: DataFrame de pandas utilizado para encontrar objetos similares dentro del mismo lugar    :param n: número de objetos para los que se buscan los vecinos más cercanos    :param k: número de vecinos más cercanos a devolver
    :param métrica: nombre de la métrica de distancia    """

    nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=k, algorithm="auto", metric=metric).fit(df[feature_names])
    nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[n][feature_names]], k, return_distance=True)
    
    df_res = pd.concat([
        df.iloc[nbrs_indices[0]], 
        pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])
        ], axis=1)
    
    return df_res

Escalar datos.

In [ ]:
# Definición de características
feature_names = ['gender', 'age', 'income', 'family_members']

# Escalado de características
transformer_mas = sklearn.preprocessing.MaxAbsScaler().fit(df[feature_names].to_numpy())

df_scaled = df.copy()
df_scaled.loc[:, feature_names] = transformer_mas.transform(df[feature_names].to_numpy())

In [ ]:
# Visualización de 5 registros aleatorios tras escalado

df_scaled.sample(5)

Ahora, vamos a obtener registros similares para uno determinado, para cada combinación

In [ ]:
# Obtención de KNN (k=5) con métrica euclidiana (datos originales)

get_knn(df, 5, 5, metric = 'euclidean')

In [ ]:
# Obtención de KNN (k=5) con métrica euclidiana (datos escalados)

get_knn(df_scaled, 5, 5, metric = 'euclidean')

In [ ]:
# Obtención de KNN (k=5) con métrica manhattan (datos originales)

get_knn(df, 5, 5, metric = 'manhattan')

In [ ]:
# Obtención de KNN (k=5) con métrica manhattan (datos escalados)

get_knn(df_scaled, 5, 5, metric = 'manhattan')

**¿El hecho de que los datos no estén escalados afecta al algoritmo kNN? Si es así, ¿cómo se manifiesta?** 

Sí, el hecho de que los datos no estén escalados **afecta significativamente** al algoritmo kNN. Esto se debe a que el algoritmo se basa en cálculos de distancia para determinar la similitud entre puntos de datos. Cuando las características tienen escalas diferentes, aquellas con magnitudes mayores dominan el cálculo de la distancia, lo que distorsiona los resultados.

**¿Qué tan similares son los resultados al utilizar la métrica de distancia Manhattan (independientemente del escalado)?** 

La distancia Manhattan a veces ***puede* producir resultados más estables** en presencia de datos no escalados o con valores atípicos. Sin embargo, si no se escalan los datos, cualquiera de las dos puede dar resultados sesgados porque las características con números más grandes dominarán la comparación. Por eso, para que el algoritmo kNN funcione bien y los resultados sean más confiables, es importante escalar los datos antes de usar cualquiera de estas distancias.

# Tarea 2. ¿Es probable que el cliente reciba una prestación del seguro?

En términos de machine learning podemos considerarlo como una tarea de clasificación binaria.

Con el valor de `insurance_benefits` superior a cero como objetivo, evalúa si el enfoque de clasificación kNN puede funcionar mejor que el modelo dummy.
Instrucciones:
- Construye un clasificador basado en KNN y mide su calidad con la métrica F1 para k=1...10 tanto para los datos originales como para los escalados. Sería interesante observar cómo k puede influir en la métrica de evaluación y si el escalado de los datos provoca alguna diferencia. Puedes utilizar una implementación ya existente del algoritmo de clasificación kNN de scikit-learn (consulta [el enlace](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)) o tu propia implementación.- Construye un modelo dummy que, en este caso, es simplemente un modelo aleatorio. Debería devolver "1" con cierta probabilidad. Probemos el modelo con cuatro valores de probabilidad: 0, la probabilidad de pagar cualquier prestación del seguro, 0.5, 1.
La probabilidad de pagar cualquier prestación del seguro puede definirse como
$$
P\{\text{prestación de seguro recibida}\}=\frac{\text{número de clientes que han recibido alguna prestación de seguro}}{\text{número total de clientes}}.
$$

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30.

In [ ]:
# Cálculo de objetivos (originales)

df['insurance_benefits_received'] = (df['insurance_benefits'] > 0).astype('int')
print('Conjunto de datos sin escalar')
display(df.head(5))

# Cálculo de objetivos (escalados)

df_scaled['insurance_benefits_received'] = (df_scaled['insurance_benefits'] > 0).astype('int')
print('Conjunto de datos escalado')
df_scaled.head(5)

In [ ]:
# Comprobación del desequilibrio de clases

# Distribución de beneficios de seguro recibidos 
imbalance = df['insurance_benefits_received'].value_counts()

# Número de personas con y sin beneficios de seguros
print('Personas SIN prestaciones de seguros: ', imbalance.get(0)) 
print('Personas CON prestaciones de seguros: ', imbalance.get(1), '\n') 

# Visualización en gráfico de barras
imbalance.plot(kind='bar')
ax = imbalance.plot(kind='bar')
ax.set_xticklabels(['SIN (0)', 'CON (1)'], rotation=0)
plt.title("Desequilibrio de clases (Acceso a prestaciones de seguro)")  # Add title to the plot
plt.show()

In [ ]:
# Evaluación del clasificador: F1 y matriz de confusión normalizada

def eval_classifier(y_true, y_pred):
    
    f1_score = sklearn.metrics.f1_score(y_true, y_pred)
    print(f'F1: {f1_score:.2f}', '\n')
    
    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='all')
    cm_formatted = np.array2string(cm, formatter={'float_kind':lambda x: f"{x:.4f}"})
    print('Matriz de confusión')
    print(cm_formatted)

In [ ]:
# Modelo de predicción binomial aleatoria

def rnd_model_predict(P, size, seed=42):

    rng = np.random.default_rng(seed=seed)
    return rng.binomial(n=1, p=P, size=size)

In [ ]:
# Evaluación del modelo aleatorio con diferentes probabilidades de predicción

for P in [0, df['insurance_benefits_received'].sum() / len(df), 0.5, 1]:

    print(f'La probabilidad: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P, size = len(df))
        
    eval_classifier(df['insurance_benefits_received'], y_pred_rnd)
    
    print('=================\n')

**División en entrenamiento y prueba**

In [ ]:
# División en conjuntos de entrenamiento y validación (datos originales)
features = df.drop(['insurance_benefits_received', 'insurance_benefits'], axis = 1)
target = df['insurance_benefits_received']
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.3, 
                                                                              random_state = 12345)

# División en conjuntos de entrenamiento y validación (datos escalados)
features_sc = df_scaled.drop(['insurance_benefits_received', 'insurance_benefits'], axis = 1)
target_sc = df_scaled['insurance_benefits_received']
features_train_sc, features_valid_sc, target_train_sc, target_valid_sc = train_test_split(features_sc, target_sc, test_size = 0.3, 
                                                                              random_state = 12345)


**Clasificador basado en kNN**

In [ ]:
# Búsqueda del mejor valor de k para KNN usando F1 score

def class_knn(features_train, features_valid, target_train, target_valid, metric):

    best_k = 0
    best_f1 = 0
    
    # Búsqueda de la mejor exactitud
    for k in range(1, 11):
        kneigh = sklearn.neighbors.KNeighborsClassifier(n_neighbors = k, weights = 'distance', metric = metric, p=2)
        kneigh.fit(features_train, target_train)
        
        predictions = kneigh.predict(features_valid)
        f1 = sklearn.metrics.f1_score(target_valid, predictions)
        print('k= ', k, 'F1 score', f1.round(4))
    
        if f1 > best_f1:
            best_f1 = f1
            best_k = k
        
    print(f'\nEl mejor F1 score es {best_f1:.4f} con k = {best_k}')

In [ ]:
# Implementación de KNN (datos originales)

class_knn(features_train, features_valid, target_train, target_valid, 'euclidean')

Con los datos no escalados, el mejor valor de F1 obtenido fue aproximadamente 0.6165 con $k=1$. Las puntuaciones F1 disminuyeron a medida que $k$ aumentaba.

In [ ]:
# Predicción y Evaluación del Clasificador KNN (k=1)

knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 1, weights = 'distance', metric = 'euclidean', p=2)
knn.fit(features_train, target_train)
        
predictions_k1 = knn.predict(features_valid)

eval_classifier(target_valid, predictions_k1)

Para los datos no escalados, la matriz de confusión muestra que el modelo clasifica correctamente el 87.13% de los casos negativos y comete un 2% de falsos positivos. En cuanto a la clase positiva, el modelo acierta solo el 5.73% de los casos y falla en un 5.13%, lo que indica una baja sensibilidad para detectar positivos. Esto refleja un desempeño limitado, con un F1 score de 0.62, donde el modelo tiene dificultad para identificar correctamente la clase minoritaria o positiva.

In [ ]:
# Implementación de KNN (datos escalados)

class_knn(features_train_sc, features_valid_sc, target_train_sc, target_valid_sc, 'euclidean')

Con los datos escalados el mejor valor de **F1 mejoró drásticamente** a aproximadamente 0.9659 con $k=1$. Las puntuaciones F1 se mantuvieron altas para $k=1$ y $k=2$, y aunque disminuyeron ligeramente para valores mayores de $k$, siguieron siendo mucho mejores que con los datos sin escalar. Esto indica que la escala hizo que las características fueran más comparables y mejoró la capacidad del clasificador para distinguir las clases.


In [ ]:
# Predicción y Evaluación del Clasificador KNN (k=1)

knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 1, weights = 'distance', metric = 'euclidean', p=2)
knn.fit(features_train_sc, target_train_sc)
        
predictions_sc_k1 = knn.predict(features_valid_sc)

eval_classifier(target_valid_sc, predictions_sc_k1)

En el caso de los datos escalados, la matriz de confusión revela una mejora significativa: el modelo clasifica correctamente el 88.87% de los negativos y reduce los falsos positivos a apenas un 0.27%. Para la clase positiva, el modelo acierta un 10.4% de los casos y comete muy pocos falsos negativos, solo un 0.47%. Estos resultados indican un aumento notable en la capacidad del modelo para reconocer ambas clases con mayor precisión y sensibilidad, reflejado en un F1 score mucho más alto, de 0.97.

Al comparar ambas matrices, se observa que **el escalado de datos reduce significativamente los errores de clasificación**. En el conjunto escalado, tanto los falsos positivos como los falsos negativos disminuyen considerablemente, lo que permite al modelo identificar con mayor exactitud las clases positiva y negativa. Además, el aumento en los verdaderos positivos y negativos refleja un mejor equilibrio y confiabilidad del clasificador. Por el contrario, el modelo con datos sin escalar presenta mayores tasas de error, especialmente en la detección de la clase positiva, limitando su efectividad. Esto demuestra claramente la importancia del escalado para mejorar el rendimiento de kNN.

# Tarea 3. Regresión (con regresión lineal)

Con `insurance_benefits` como objetivo, evalúa cuál sería la RECM de un modelo de regresión lineal.

Construye tu propia implementación de regresión lineal. Para ello, recuerda cómo está formulada la solución de la tarea de regresión lineal en términos de LA. Comprueba la RECM tanto para los datos originales como para los escalados. ¿Puedes ver alguna diferencia en la RECM con respecto a estos dos casos?

Denotemos- $X$: matriz de características; cada fila es un caso, cada columna es una característica, la primera columna está formada por unidades- $y$ — objetivo (un vector)- $\hat{y}$ — objetivo estimado (un vector)- $w$ — vector de pesos
La tarea de regresión lineal en el lenguaje de las matrices puede formularse así:
$$
y = Xw
$$

El objetivo de entrenamiento es entonces encontrar esa $w$ w que minimice la distancia L2 (ECM) entre $Xw$ y $y$:

$$
\min_w d_2(Xw, y) \quad \text{or} \quad \min_w \text{MSE}(Xw, y)
$$

Parece que hay una solución analítica para lo anteriormente expuesto:
$$
w = (X^T X)^{-1} X^T y
$$

La fórmula anterior puede servir para encontrar los pesos $w$ y estos últimos pueden utilizarse para calcular los valores predichos
$$
\hat{y} = X_{val}w
$$

Divide todos los datos correspondientes a las etapas de entrenamiento/prueba respetando la proporción 70:30. Utiliza la métrica RECM para evaluar el modelo.

In [ ]:
# Función de regresión lineal simple

class MyLinearRegression:
    
    def __init__(self):
        self.weights = None
    
    def fit(self, X, y):
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        self.weights = np.linalg.inv(X2.T @ X2) @ X2.T @ y

    def predict(self, X):
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        y_pred = X2 @ self.weights
        return y_pred

In [ ]:
# Evaluación de un modelo de regresión: Cálculo de RMSE y R2

def eval_regressor(y_true, y_pred):
    
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    print(f'RMSE: {rmse:.2f}')
    
    r2_score = math.sqrt(sklearn.metrics.r2_score(y_true, y_pred))
    print(f'R2: {r2_score:.2f}')    

In [ ]:
# Implementación de la función de regresión lineal

def my_lr_ap(x, y):
    
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=12345)
    
    lr = MyLinearRegression()
    lr.fit(X_train, Y_train)
    print(lr.weights)

    Y_test_pred = lr.predict(X_test)
    print('Valores predichos', Y_test_pred)
    eval_regressor(Y_test, Y_test_pred)

In [ ]:
# Entrenamiento y evaluación con datos originales

features = df[['age', 'gender', 'income', 'family_members']].to_numpy()
target = df['insurance_benefits'].to_numpy()

my_lr_ap(features, target)

In [ ]:
# Entrenamiento y evaluación con datos escalados

features_sc= df_scaled[['age', 'gender', 'income', 'family_members']].to_numpy()
target_sc = df_scaled['insurance_benefits'].to_numpy()

my_lr_ap(features_sc, target_sc)

En ambos casos se obtienen resultados similares porque la regresión lineal, al incluir un término de intercepto y ajustar los coeficientes, es invariante frente al escalado de las variables predictoras. Esto significa que aunque los pesos cambien al modificar la escala de los datos, las predicciones del modelo permanecen prácticamente iguales, y por tanto las métricas de evaluación como el RMSE y el R² también resultan similares. Por ello, el escalado no afecta la precisión ni el ajuste del modelo en este contexto específico.

# Tarea 4. Ofuscación de datos

Lo mejor es ofuscar los datos multiplicando las características numéricas (recuerda que se pueden ver como la matriz $X$) por una matriz invertible $P$. 

$$
X' = X \times P
$$

Trata de hacerlo y comprueba cómo quedarán los valores de las características después de la transformación. Por cierto, la propiedad de invertibilidad es importante aquí, así que asegúrate de que $P$ sea realmente invertible.

Puedes revisar la lección 'Matrices y operaciones matriciales -> Multiplicación de matrices' para recordar la regla de multiplicación de matrices y su implementación con NumPy.

In [ ]:
# Selección de columnas de información personal

personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]
display(df_pn.head())

In [ ]:
# Conversión del df a matriz NumPy

X = df_pn.to_numpy()
display(X)

Generar una matriz aleatoria $P$.

In [ ]:
# Generación de matriz aleatoria P

rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

Comprobar que la matriz P sea invertible

In [ ]:
# Generación de matriz inversa P^-1

np.linalg.inv(P)

In [ ]:
# Transposición de datos X·P

X_trans = X @ P
df_pn_trans = pd.DataFrame(X_trans, columns = personal_info_column_list)
df_pn_trans.head()

**¿Puedes adivinar la edad o los ingresos de los clientes después de la transformación?**

No, después de la transformación no se puede adivinar la edad o los ingresos. La matriz que usaron mezcla todas las columnas originales en nuevas combinaciones, así que los números que ves ya no representan directamente ninguna variable como edad o ingreso. Sin la matriz original para "deshacer" la transformación, no hay forma de saber los valores reales.

**¿Puedes recuperar los datos originales de $X'$ si conoces $P$?** Intenta comprobarlo a través de los cálculos moviendo $P$ del lado derecho de la fórmula anterior al izquierdo. En este caso las reglas de la multiplicación matricial son realmente útiles

In [ ]:
# Recuperación de datos mediante matriz inversa

X_recover = X_trans @ np.linalg.inv(P)
df_pn_recover = pd.DataFrame(X_recover, columns = personal_info_column_list)

Muestra los tres casos para algunos clientes
- Datos originales
- El que está transformado
- El que está invertido (recuperado)

In [ ]:
# Muestra de resultados

print('Datos originales')
display(df_pn.head(3))

print('\nDatos transformados')
display(df_pn_trans.head(3))

print('\nDatos recuperados')
display(df_pn_recover.head(3))

print('\nDatos recuperados (redondeados)')
display(df_pn_recover.head(3).round(2))

**Seguramente puedes ver que algunos valores no son exactamente iguales a los de los datos originales. ¿Cuál podría ser la razón de ello?**

La razón por la que algunos valores no coinciden exactamente con los datos originales es que la matriz P transforma y mezcla las columnas en nuevas combinaciones lineales. Por eso, los números ya no representan directamente las variables originales, y pequeñas imprecisiones numéricas pueden causar diferencias. Aun así, si los redondeamos a dos decimales, podemos aproximar bastante bien los valores originales.

# Conclusiones

- El análisis preliminar de los datos exhibió una muestra equilibrada en género y una población joven, con una edad media cercana a los 31 años. Los ingresos se concentraron en niveles medios-bajos, mientras que el tamaño de los hogares fue pequeño, predominando las personas que viven solas o en pareja. Destacó la baja cobertura de prestaciones de seguro, ya que la gran mayoría carecía de acceso a estos beneficios.

- **Tarea 1:** Se desarrolló una función para identificar $k$ vecinos cercanos usando distancias euclidiana y Manhattan, observándose que la falta de escalado afectó significativamente el desempeño del algoritmo kNN. Las características con magnitudes mayores dominaron los cálculos, distorsionando los resultados, aunque Manhattan mostró cierta estabilidad ante datos no escalados o atípicos. Sin embargo, ambas métricas requirieron escalar los datos para evitar sesgos y garantizar resultados confiables.

- **Tarea 2:** Se construyó un clasificador KNN y se evaluó su desempeño con y sin escalado de datos. Los resultados mostraron que el escalado mejoró drásticamente la capacidad del modelo para distinguir entre clases, aumentando el F1 score de 0.62 a 0.97 y reduciendo significativamente los errores de clasificación, especialmente en la detección de la clase positiva. Esto evidenció que el escalado es crucial para optimizar el rendimiento del KNN en este problema.

- **Tarea 3:** Se construyó una implementación propia de regresión lineal y se observó que, aunque los coeficientes variaron al escalar las variables predictoras, las predicciones y métricas como RMSE y R² se mantuvieron prácticamente iguales. Esto confirmó que el escalado no afectó la precisión ni el ajuste del modelo en este caso.

- **Tarea 4:** Se ofuscaron los datos originales mediante una transformación lineal que mezcló todas las columnas en nuevas combinaciones, lo que impidió identificar directamente variables como edad o ingresos. Al utilizar la matriz inversa, no fue posible recuperar los valores reales con total exactitud, aunque al redondear a dos decimales se logró una aproximación bastante cercana. Las pequeñas diferencias se debieron a imprecisiones numéricas inherentes a la transformación.

- Se comprobó que al ofuscar los datos multiplicando la matriz original por una matriz invertible, los coeficientes del modelo se ajustaron de forma inversa, lo que mantuvo intactas las predicciones. Esto implicó que la RECM y el R² no variaron entre los datos originales y los ofuscados, confirmando que la calidad del modelo permaneció igual a pesar de la transformación.